In [216]:
d:"0123456789ABCDEF"!(0000b;0001b;0010b;0011b;0100b;0101b;0110b;0111b;1000b;1001b;1010b;1011b;1100b;1101b;1110b;1111b)
toBits:{raze x each y}[d]
d

0| 0000b
1| 0001b
2| 0010b
3| 0011b
4| 0100b
5| 0101b
6| 0110b
7| 0111b
8| 1000b
9| 1001b
A| 1010b
B| 1011b
C| 1100b
D| 1101b
E| 1110b
F| 1111b


In [217]:
literal:toBits "D2FE28"
literal

110100101111111000101000b


In [219]:
/ return list:
/ 0) index
/ 1) version
/ 2) type
/ 3) value

/ parse sequentially, return list of 4 above for each call
pp:{[whole;idx]
    if[not any idx _ whole;
        show"Exhausted packet";
        :enlist (idx;0;0Nj;0Nj);
     ];    
     
    result:();
    version:2 sv (idx;3) sublist whole;
    idx+:3;
    typeId:2 sv (idx;3) sublist whole;
    idx+:3;
    if[typeId=4; / literal
        l:();
        while[first b5:(idx;5) sublist whole;
            idx+:5;
            l,:1_b5
         ];
        l,:1_b5;
        idx+:5;
        v:2 sv l;        
        /show "Literal: ",string v;        
        result,:enlist (idx;version;typeId;v);
     ];
    if[typeId<>4; / operator
        lengthTypeId:whole idx;
        idx+:1;
        if[not lengthTypeId; / 15 bit of payload length
            len:2 sv (idx;15) sublist whole;
            idx+:15;
            /show "Operator type ",string[typeId]," LT=0: will parse ",string[len]," bits";         
            sbpck:();
            midx:idx+len;
            while[idx<midx;
                pck:pp[whole;idx];
                idx:first first pck;
                sbpck,:pck;
             ];
            result,:enlist (idx;version;typeId;sbpck);
          ];
        if[lengthTypeId; / 11 bit of num of sub-packed
            nsp:2 sv (idx;11) sublist whole;
            idx+:11;
            /show "Operator type ",string[typeId]," LT=1: will parse ",string[nsp]," sub-packets";
            sbpck:();
            do[nsp;
                pck:pp[whole;idx];
                idx:first first pck;
                sbpck,:pck;
             ];
            result,:enlist (idx;version;typeId;sbpck);
         ];
     ];
    
    result
 };

In [220]:
.Q.s1 pp[;0] toBits "EE00D40C823060"

",(51;7;3;(29 2 4 1;40 4 4 2;51 1 4 3))"


In [221]:
.Q.s1 pp[;0] literal

",21 6 4 2021"


In [222]:
.Q.s1 pp[;0] toBits "38006F45291200"

",(49;1;6;(33 6 4 10;49 2 4 20))"


In [223]:
sumVersions:{$[0>type last x;x 1;x[1]+sum sumVersions each x 3]};

In [224]:
16=sumVersions first pp[;0] toBits "8A004A801A8002F478"

1b


In [225]:
12=sumVersions 0N!first pp[;0] toBits "620080001611562C8802118E34"

(102;3;0;((62;0;0;(51 0 4 10;62 5 4 11));(102;1;0;(91 0 4 12;102 3 4 13))))


1b


In [226]:
23=sumVersions 0N!first pp[;0] toBits "C0015000016115A2E0802F182340"

(106;6;0;((66;0;0;(55 0 4 10;66 6 4 11));(106;4;0;(95 7 4 12;106 0 4 13))))


1b


In [227]:
31=sumVersions 0N!first pp[;0] toBits "A0016C880162017C3686B18A3D4780"

(113;5;0;,(113;1;0;,(113;3;0;(69 7 4 6;80 6 4 6;91 5 4 12;102 2 4 15;113 2 4 15))))


1b


In [228]:
input:first read0 `:input.txt
input

"220D700071F39F9C6BC92D4A6713C737B3E98783004AC0169B4B99F93CFC31AC4D8A4BB89E9D654D216B80131DC0050B20043E27C1F83240086C468A311CC0188DB0BA12B00719221D3F7AF776DC5DE635094A7D2370082795A52911791ECB7EDA9CFD634BDED14030047C01498EE203931BF7256189A593005E116802D34673999A3A805126EB2B5BEEBB823CB561E9F2165492CE00E6918C011926CA005465B0BB2D85D700B675DA72DD7E9DBE377D62B27698F0D4BAD100735276B4B93C0FF002FF359F3BCFF0DC802ACC002CE3546B92FCB7590C380210523E180233FD21D0040001098ED076108002110960D45F988EB14D9D9802F232A32E802F2FDBEBA7D3B3B7FB06320132B0037700043224C5D8F2000844558C704A6FEAA800D2CFE27B921CA872003A90C6214D62DA8AA9009CF600B8803B10E144741006A1C47F85D29DCF7C9C40132680213037284B3D488640A1008A314BC3D86D9AB6492637D331003E79300012F9BDE8560F1009B32B09EC7FC0151006A0EC6082A0008744287511CC0269810987789132AC600BD802C00087C1D88D05C001088BF1BE284D298005FB1366B353798689D8A84D5194C017D005647181A931895D588E7736C6A5008200F0B802909F97B35897CFCBD9AC4A26DD880259A0037E49861F4E4349A6005CFAD180333E95281338A930EA400824981

In [229]:
/part 1
sumVersions first parsed:pp[;0] toBits input

1014


In [230]:
/ return list:
/ 0) index
/ 1) version
/ 2) type
/ 3) value

types:0 1 2 3 5 6 7!(sum;prd;min;max;>;<;=);
build:{
    if[7h=type x;:x 3]; / literal
    
    typeId:x 2;op:types typeId;
    if[typeId in 5 6 7;:(op;build first x 3;build last x 3)]; / binary operators
    
    :(op;(enlist,build each x 3)) / list operators
 };

In [231]:
build first pp[;0] literal

2021


In [232]:
eval 0N!build first pp[;0] toBits "EE00D40C823060"

(max;(enlist;1;2;3))


3


In [233]:
/ part 2
eval build first parsed

1922490999789
